In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(labels = ['EIN', 'NAME'], axis = 1, inplace = True)

In [4]:
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = application_df.APPLICATION_TYPE.value_counts()\
.loc[application_df.APPLICATION_TYPE.value_counts() < 50].index

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [98]:
application_df.CLASSIFICATION.value_counts().head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = application_df.CLASSIFICATION.value_counts()\
.loc[application_df.CLASSIFICATION.value_counts() < 100].index

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
# STATUS and IS_SUCCESSFUL are also categorical but are already encoded
application_cat=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE',
                 'ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(X=application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/jsaizlomas/miniforge3/envs/propensity2sell/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
encode_df.shape

(34299, 49)

In [10]:
#§ Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df.drop(labels=application_cat,axis=1), encode_df], axis=1)

In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(labels=['IS_SUCCESSFUL'],axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
X_train_scaled.shape

(22980, 51)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(80, activation='relu'))
nn.add(tf.keras.layers.Dense(80, activation='relu'))
nn.add(tf.keras.layers.Dense(80, activation='relu'))
nn.add(tf.keras.layers.Dense(20, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))
nn.build(X_train_scaled.shape)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (22980, 80)               4160      
                                                                 
 dense_1 (Dense)             (22980, 80)               6480      
                                                                 
 dense_2 (Dense)             (22980, 80)               6480      
                                                                 
 dense_3 (Dense)             (22980, 20)               1620      
                                                                 
 dense_4 (Dense)             (22980, 1)                21        
                                                                 
Total params: 18,761
Trainable params: 18,761
Non-trainable params: 0
_________________________________________________________________


2022-06-28 11:42:03.816582: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [17]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="saved_models_opt/weights.{epoch:02d}.hdf5",
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [18]:
# Train the model
nn.fit(x=X_train_scaled, batch_size=16, y=y_train,validation_data=(X_test_scaled, y_test), 
       epochs=100, callbacks=[cp_callback])

Epoch 1/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5692 - accuracy: 0.7203 - val_loss: 0.5557 - val_accuracy: 0.7262
Epoch 2/100
1437/1437 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7279 - val_loss: 0.5523 - val_accuracy: 0.7305
Epoch 3/100
1437/1437 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7309 - val_loss: 0.5550 - val_accuracy: 0.7297
Epoch 4/100
1437/1437 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7329 - val_loss: 0.5539 - val_accuracy: 0.7322
Epoch 5/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5467 - accuracy: 0.7324 - val_loss: 0.5533 - val_accuracy: 0.7298
Epoch 6/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5460 - accuracy: 0.7332 - val_loss: 0.5560 - val_accuracy: 0.7187
Epoch 7/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5446 - accuracy: 0.7325 - val_loss: 0.5548 - val_ac

1437/1437 [==============================] - 2s 1ms/step - loss: 0.5290 - accuracy: 0.7418 - val_loss: 0.5802 - val_accuracy: 0.7299
Epoch 58/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5292 - accuracy: 0.7411 - val_loss: 0.5743 - val_accuracy: 0.7304
Epoch 59/100
1437/1437 [==============================] - 1s 1ms/step - loss: 0.5289 - accuracy: 0.7404 - val_loss: 0.5792 - val_accuracy: 0.7301
Epoch 60/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5294 - accuracy: 0.7413 - val_loss: 0.5935 - val_accuracy: 0.7297
Epoch 61/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5286 - accuracy: 0.7412 - val_loss: 0.5805 - val_accuracy: 0.7308
Epoch 62/100
1437/1437 [==============================] - 2s 1ms/step - loss: 0.5287 - accuracy: 0.7411 - val_loss: 0.5899 - val_accuracy: 0.7305
Epoch 63/100
1437/1437 [==============================] - 1s 1ms/step - loss: 0.5285 - accuracy: 0.7418 - val_loss: 0.6024 - val_accuracy

In [129]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

354/354 - 0s - loss: 0.5628 - accuracy: 0.7294 - 173ms/epoch - 488us/step
Loss: 0.5628463625907898, Accuracy: 0.7293930649757385


In [131]:
plt.plot(nn.history.history["accuracy"])
plt.plot(nn.history.history["val_accuracy"])

KeyError: 'accuracy'

In [132]:
nn.history.history

{}